## Pre-processing

In [3]:
import csv

data_list = []

csv_file_path = "furniture_stores_pages.csv"

# Open the CSV file and read its contents
with open(csv_file_path, "r") as csv_file:
    csv_reader = csv.reader(csv_file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        data_list.append(row[0])

In [4]:
data_list[:5]

['max(page)',
 'https://www.factorybuys.com.au/products/euro-top-mattress-king',
 'https://dunlin.com.au/products/beadlight-cirrus',
 'https://themodern.net.au/products/hamar-plant-stand-ash',
 'https://furniturefetish.com.au/products/oslo-office-chair-white']

In [5]:
data_list = data_list[1:]

seperate the webpages into:
- non existend content
- content exists but there is no main section in html
- content with main section

In [6]:
import requests
from bs4 import BeautifulSoup

def extract_main_content(urls):
  not_exits,main_not_exixst,main_text = [],[],[]
  for url in urls:

    try:
      response = requests.get(url)

        # Check if the request was successful
      if response.status_code == 200:
          # Parse the HTML content of the webpage
          soup = BeautifulSoup(response.text, 'html.parser')

          # Find the main content by inspecting the HTML structure of the webpage
          # and locating the relevant tags or classes
          main_content = soup.find('main')

          # Extract the text from the main content
          if main_content:
            main_text.append(main_content.get_text())
          else:
            main_not_exixst.append(url)
      else:
          not_exits.append(url)
    except:
      not_exits.append(url)

  return not_exits,main_not_exixst,main_text

In [7]:
not_exits,main_not_exixst,main_text = extract_main_content(data_list)

In [32]:
main_text[5]

'\n\n\n\n\n\nHoyt Chair\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n View in AR\n\n\n\n\nHoyt Chair\n\n\n\nRegular price\n\n          $1,450.00\n        \n\n\n\n          Prima Linen - $1,450.00 USD\n        \n\n          Sophie Sky Grey - $1,480.00 USD\n        \n\n          Ease Silver Dollar - $1,450.00 USD\n        \n\n          Simplex Oyster - $1,450.00 USD\n        \n\n          Taylor Felt Grey - $1,450.00 USD\n        \n\n          Vibes Elephant Grey - $1,495.00 USD\n        \n\n          Romo Moon - $1,510.00 USD\n        \n\n          Notion Underground - $1,450.00 USD\n        \n\n          Twist Earl Grey - $1,450.00 USD\n        \n\n          Notion Airstream - $1,450.00 USD\n        \n\n          Vibes Steel - $1,495.00 USD\n        \n\n          Bennett Charcoal - $1,450.00 USD\n        \n\n          Romo Lead - $1,510.00 USD\n        \n\n          Poodle Cloud - $1,510.00 USD\n        \n\n          Nubia Linen - $1,555.00 USD\n        \n\n          Eton Ivory - $1,450.00

In [263]:
#clean text
import re
main_text_clean = [re.sub('(\n+)(?=\n)','',txt) for txt in main_text]
main_text_clean = [re.sub('\n',' ',txt) for txt in main_text_clean]
main_text_clean = [re.sub(r'\s{2,}',' ',txt) for txt in main_text_clean]

In [264]:
main_text_clean[2]

' Home / Gift Cards Gift Cards Filter By {filter.label} (1) Selected: { value.label } ({value.count}) $ From $ To Apply All filters Filter by Filter by: products Clear all filters {filter.label} (1) { value.label } ({value.count}) {filter.label} (1) $ From $ To Clear all Apply () Apply products Sort by Sort Vendor: RJ Living $100 RJ Living Gift Card Vendor: RJ Living $100 RJ Living Gift Card Vendor: RJ Living $100 RJ Living Gift Card Vendor: RJ Living $100 RJ Living Gift Card Vendor: RJ Living $100 RJ Living Gift Card Vendor: RJ Living $100 RJ Living Gift Card Vendor: RJ Living $100 RJ Living Gift Card You are viewing out of products Load more products You may also like Save for later Vendor: RJ Living Sector Coffee Table - Earth Marble 4 colours Regular price $1,019.15 Regular price $1,199 Sale price $1,019.15 Unit price / per SALE 15% OFF Save for later Vendor: Saardé Flow Resin Salad Server - Earth 4 colours Regular price $50.15 Regular price $59 Sale price $50.15 Unit price / per S

In [266]:
#save for future use
import pickle
with open('not_exits.pkl', 'wb') as f:
    pickle.dump(not_exits, f)
    
with open('main_not_exixst.pkl', 'wb') as f:
    pickle.dump(main_not_exixst, f)

with open('main_text_clean.pkl', 'wb') as f:
    pickle.dump(main_text_clean, f)

We will use chat gpt to inspect the text from the webpage and return a list of product names from it (RAG inspired). We will use this as the tags for the training and validation data.

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.environ['OPEN_AI_API_KEY'])

In [199]:
import ast

products_list = []
for text in main_text_clean:
    prompt = f'''you will be provided with text extracted from a furniture store wesbite.
This text will contain product names.
Please provide a list of the product names found in the text.
Give the list as a python list, if there are no products give an empty list.
Please do not repeat items. Text: {text} Python list:''' 
    
    try:
        res = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
          ]
        )

        content = res.choices[0].message.content
        try:
            p_lst = re.search(r'\[.*?\]',content,flags=re.DOTALL).group()
        except:
            p_lst = '[]'
        p_lst = ast.literal_eval(p_lst)
        products_list.append(p_lst)
    except:
        products_list.append(None)

In [206]:
for i,l in enumerate(products_list):
    if l==[] or l==None:
        print(i,l,main_text_clean[i])

62 []  Email Spread the word Share Share on Facebook Tweet Tweet on Twitter Pin it Pin on Pinterest 
91 []  Welcome to Royal GardenIf you have a password to enter the site, please click the upper right hand link above to enter it. Wholesale Sign-up Loading… 
130 [] 
147 []  


In [231]:
with open('products_list.pkl','wb') as f:
    pickle.dump(products_list,f)

In [307]:
with open('products_list.pkl','rb') as f:
    products_list = pickle.load(f)
    
with open('main_text_clean.pkl','rb') as f:
    main_text_clean = pickle.load(f)

In [308]:
main_text_clean[145]

" View in space Sale Sold Out Industrial 2x Handmade X Dining Table Bench Frame Steel Table Pedestal Legs TL009 2 reviews RusticLand £95.97 Unit price / per Size 35 x 40cm Bench 50 x 71cm Table 60 x 71cm Table 70 x 71cm Table Colour Raw Steel Black 35 x 40cm Bench / Raw Steel 35 x 40cm Bench / Black 50 x 71cm Table / Raw Steel 50 x 71cm Table / Black 60 x 71cm Table / Raw Steel 60 x 71cm Table / Black 70 x 71cm Table / Raw Steel 70 x 71cm Table / Black Quantity Cart Error Add to Cart Added Cart Error Add to Cart Added Sold Out Social Sharing Share on Facebook Share on Twitter Share on Pinterest 30315379523686 95.97 30315379654758 115.97 30315379556454 127.97 30315379687526 147.97 30315379589222 149.97 30315379720294 169.97 30315379621990 169.97 30315379753062 189.97 Introducing our Heavy Duty Industrial X Style Table Legs, designed to provide robust support and a stylish industrial aesthetic for your table. These table legs feature a strong steel construction with a 40x40mm box section

Now we will make two lists:
- texts which contains lists of the words in a webpage text 
- tags which contains lists of tags, tagging each word in the webpage text

**We use IOB method for tagging**

In [309]:
texts = [re.split(',|;| ',text) for text in main_text_clean]
tags = []

for text,products in zip(texts,products_list):
    tags_l = ['O' for _ in range(len(text))]
    for product in products:
        start_word = product.split()[0]
        word_len = len(product.split())
        #print(product)
        for i in range(len(text)):
            if start_word in text[i]:
                if product in ' '.join(text[i:i+word_len]):
                    if word_len>1:
                        tags_l[i] = 'B-PRODUCT' 
                        tags_l[i+1:i+word_len] = ['I-PRODUCT']*(word_len-1)
                        #print(text[i],tags_l[i])
                        #print(text[i+1:i+word_len],tags_l[i+1:i+word_len])
                    else:
                        tags_l[i] = 'B-PRODUCT'
    
    tags.append(tags_l)

In [310]:
for i in range(len(tags[147])):
    print(texts[147][i],tags[147][i])

 O
 O


In [282]:
for i in range(len(texts)):
    if len(texts[i]) == 0:
        print(i,tags[i])

In [311]:
#save these for use when constructing the dataset for training
with open('texts.pkl','wb') as f:
    pickle.dump(texts,f)
with open('tags.pkl','wb') as f:
    pickle.dump(tags,f)

## Inference

In [1]:
import torch

In [2]:
#load saved fine tuned model
from transformers import AutoModelForTokenClassification,AutoTokenizer
model = AutoModelForTokenClassification.from_pretrained('./fine-tuned-ner')
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-ner')

In [3]:
import pickle
with open('main_text_clean.pkl','rb') as f:
    main_text_clean = pickle.load(f)
with open('main_not_exixst.pkl','rb') as f:
    main_not_exixst = pickle.load(f)

In [4]:
test = main_text_clean[30]

In [5]:
test

" Vintage Leather & Harris Tweed Our Vintage Leather is the highest quality, aniline leather; We only use top quality, 100% natural leather and it is completely developed and coloured by hand from start to finish. This is the only way to get this totally authentic aged look. There are no short cuts here - and it results in beautiful, deep tones and patinas, aged to perfection and totally unique. Explore The Collection Fabric Sofas Soft to the touch and impossible to resist, our beautiful collection of fabric sofas has something for everyone, from stylish and contemporary, to elegant and classic. Explore The Collection Leather Sofas Leather sofas crafted from genuine leather hides, Our collection bring's a touch of class to your living space; from chic Italian to contemporary British designs, traditional Chesterfields & luxury designer brands. Explore The Collection Recliner Sofas Recliner sofas are just the thing to come home to after a long hard day - put your feet up and enjoy the ul

In [6]:
inputs = tokenizer(test,return_tensors='pt',truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

In [7]:
predicted = outputs.logits.argmax(dim=-1)

In [8]:
def get_products_list(inputs,predicted):

    met_word = False
    prods = []
    prev_word_id = None
    for i,label in enumerate(predicted[0]):
        word_id = inputs.word_ids(batch_index=0)[i]
        if label == 1 and not met_word:
            prod_start = i
            met_word = True
        elif label == 1 and met_word and prev_word_id!=word_id:
            prod_end = i
            prod = tokenizer.decode(inputs['input_ids'][0][prod_start:prod_end])
            prods.append(prod)
            prod_start = i
        elif label == 0 and met_word:
            prod_end = i
            prod = tokenizer.decode(inputs['input_ids'][0][prod_start:prod_end])
            prods.append(prod)
            met_word = False

        prev_word_id = word_id
    return prods
        


In [9]:
get_products_list(inputs,predicted)

['##tage', 'Leather & Harris Tweed Our', 'So']

Write function to get url and return a list of product names

In [10]:
import re
def clean_text(text):
    clean = re.sub('(\n+)(?=\n)','',text) #remove all \n except last one
    clean = re.sub('\n',' ',clean) #replace last \n with ' '
    clean = re.sub(r'\s{2,}',' ',clean) #replace two or more consecutive ' ' with one ' '
    return clean

In [11]:
import requests
from bs4 import BeautifulSoup

def get_products_list_from_url(url):
    prods_list = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        main_content = soup.find('main')
        if main_content:
            text = main_content.get_text()
            text = clean_text(text)
        else:
            text = soup.find().get_text()
            text = clean_text(text)
            
        inputs = tokenizer(text,return_tensors='pt',truncation=True) #truncate if over 512 max tokens for model
        with torch.no_grad():
            outputs = model(**inputs)
        predicted = outputs.logits.argmax(dim=-1)
        
        return (get_products_list(inputs,predicted),text)
    except:
        return [],''

In [24]:
prods, txt = get_products_list_from_url(main_not_exixst[27])

In [25]:
prods

['Albini',
 '& Franca Helg Jin Kuramoto',
 'Buffet Nanna',
 '& Jørgen',
 'Ditzel Robert',
 'W. Tove Kindt - Larsen V',
 'Bo',
 'Albin',
 'Ku',
 'Buffet',
 'Nanna',
 '& Jørgen',
 'Ditzel Robert',
 'W. Tove Kindt - Larsen V',
 'Bo',
 '##n',
 'Furn',
 'Chairs Dining',
 'Tables',
 'Lounge',
 'Chairs Swings Sofas Coffee Tables Ottomans',
 '& Poufs Headboards',
 '##s Benches Bar',
 '& Foot',
 'Stools Bar Trolleys',
 'Shelves',
 'Chairs',
 'Garden',
 'Tables Lounge',
 'Chairs Swings Sunbeds Sofas Coffee',
 'Tables',
 '& Stools Bar Trolley']

In [26]:
txt

" All Products - Sika-Design.com Store locator Catalogues Press Downloads B2B Shop 0 Cart Products Designers Arne Jacobsen Franco Albini & Franca Helg Jin Kuramoto Michel Buffet Nanna & Jørgen Ditzel Robert W. Tove Kindt-Larsen Viggo Boesen Yuzuru Yamakawa For professionals B2B Shop Downloads Partners & agents New B2B customer Projects Events Catalogues Press About us Our history Responsibility What is Rattan? Our Partners Store locator Contact Our team Jobs & careers Responsibility Products Designers Arne Jacobsen Franco Albini & Franca Helg Jin Kuramoto Michel Buffet Nanna & Jørgen Ditzel Robert W. Tove Kindt-Larsen Viggo Boesen Yuzuru Yamakawa For professionals B2B Shop Downloads Partners & agents New B2B customer Projects Events Catalogues Press About us Our history Responsibility What is Rattan? Our Partners Store locator Contact Our team Jobs & careers Responsibility Search 0 Cart Indoor Furniture Dining Chairs Dining Tables Lounge Chairs Swings Sofas Coffee Tables Ottomans & Pou

## Next steps and improvements

- add evaluation metric on validation data to see how well model performs. At the moment i only used training loss to gauge the performance however precision,recall,f1 should also be computed.
- deal with texts which go over model limit. A solution would be to break the texts into smaller chunks in case they are too big and then use those for training (this way we dont lose any data and will have more training data), also for inference we could do the same and feed to the model more chunks broken from a single webpage.
- make interface with gradio
- deal with product list not always containing words. There are cases when model might predict a sub token as being a single product, these are mistakes and should be excluded from the list returned.
- examine the webpages a bit more and maybe find a better way to extract relevant text

## Struggles 

- At first i wasant sure how to tag sample texts to be used for training. In the end i chose to use llm to give me a list of the products found in the text and then use that list to tag each word. This turned out to be a pretty good method and i was able to produce a reliable dataset. I also chose to use the webpages which had a clear indication of what the main section is in the html, thankfully these were plenty for training.
- I also struggled with training. At first i tried using the high level Trainer class from huggingface, however after training for a couple of epochs the model was not predicting anything and it was telling me that there was no log of the training loss. I then also tried training with my own training loop and printed the loss at the end of every epoch, i have seen that the loss is indeed decreasing which made me realise that i need to train for way more epochs than i first did. Therefore i believe my mistake was not training for long enough which made the Trainer class not display accurate metrics for the first few epochs.

## Learning outcomes

- what NER is
- how to use BeautifulSoup to extract text from webpages
- how to fine tune a NER model from hugging face 
- how to construct custom data sets compatible with hugging face (for NER)